In [1]:
# !pip install numpy
# !pip install numpy-stl
# !pip install tripy
# !pip install plotly

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from stl import mesh
# from google.colab import drive
import numpy as np

In [3]:


# drive.mount('/content/drive')
# drive_path = "/content/drive/MyDrive/Research/PhD/Scripts/"
# stl_file_path = drive_path + "test_part.stl"
# stl_file_path = "./squares.stl"
# stl_file_path = "./vessel_half.stl"
# stl_file_path = "./circle_split.stl"
# stl_file_path = "./vessel_quarter.stl"
# stl_file_path = "./simple_split.stl"
# stl_file_path = "./neuro_model.stl"
stl_file_path = "./neuro_model_half_rotated.stl"





mesh_data = mesh.Mesh.from_file(stl_file_path)

# Extract vertices
vertices = mesh_data.vectors.reshape((-1, 3))

# Create a 3D scatter plot
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

fig.add_trace(go.Scatter3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    mode='markers',
    marker=dict(size=2),
))

# Set layout
fig.update_layout(scene=dict(aspectmode="data"))

# Show the plot
fig.show()

In [4]:
# def on_top_surface(triangle, top_height):
#   return np.all(triangle[:,2]>top_height)

def is_vertical(triangle):
  return not np.allclose(triangle[:, 2], triangle[0, 2], atol = 1e-4)

def has_top_edge(triangle, top_height):
  return np.sum(triangle[:, 2] > top_height) == 2
  

In [5]:
print(vertices.shape)
np.set_printoptions(suppress=True, precision=2)

height_tolerance = 1 #mm

# print(vertices)
# n_scores[:, 1:-1].max(axis=1)
# print("max z height", str(vertices[:,2].max()))

# height threshold, accounting for tolerance
height_thresh = vertices[:,2].max()-height_tolerance

triangles =  np.array([vertices[0:3,:]])

# organizes vertices into triangles
for i in range(int(vertices.shape[0]/3)):
  temp_tri = np.array([vertices[3*i:3*(i+1),:]])
  triangles = np.vstack((triangles, temp_tri))


# extracts top surface triangles that have a top edge.
vertical_tris = np.array([])
for ind in range(triangles.shape[0]):
  # checks if on top surface
  if(is_vertical(triangles[ind]) and has_top_edge(triangles[ind], height_thresh)):
    if(vertical_tris.size == 0):
      vertical_tris = np.array([triangles[ind]])
      print("triangle start shape", str(vertical_tris.shape))
    else:
      vertical_tris = np.vstack((vertical_tris, [triangles[ind]]))

print(vertical_tris.shape[0], "vertical triangles on top edge")
print(vertical_tris)


(3612, 3)
triangle start shape (1, 3, 3)
301 vertical triangles on top edge
[[[275.92 853.98   6.  ]
  [279.81 855.13   2.  ]
  [279.81 855.13   6.  ]]

 [[266.67 852.51   6.  ]
  [271.51 853.07   2.  ]
  [271.51 853.07   6.  ]]

 [[273.03 850.59   6.  ]
  [267.83 849.89   2.  ]
  [267.83 849.89   6.  ]]

 ...

 [[270.29 855.45   6.  ]
  [252.4  855.52   2.  ]
  [252.4  855.52   6.  ]]

 [[279.81 855.13   6.  ]
  [270.29 855.45   2.  ]
  [270.29 855.45   6.  ]]

 [[271.51 853.07   6.  ]
  [275.92 853.98   2.  ]
  [275.92 853.98   6.  ]]]


In [6]:
def get_top_edge(triangle, top_height):
	edge = triangle[triangle[:, 2] > top_height]
	return edge

# print(get_top_edge(vertical_tris[0], height_thresh))

#extracts the surface edge from the vertical triangles
surface_lines = np.array([])

for ind in range(vertical_tris.shape[0]):
	if(surface_lines.size == 0):
		surface_lines = np.array([get_top_edge(vertical_tris[ind], height_thresh)])
		print("triangle start shape", str(surface_lines.shape))
	else:
		# print(surface_lines.shape)
		# print(get_top_edge(vertical_tris[ind], height_thresh).shape)
		# print(get_top_edge(vertical_tris[ind], height_thresh))
		surface_lines = np.vstack((surface_lines, [get_top_edge(vertical_tris[ind], height_thresh)]))

print(surface_lines.shape)
print(surface_lines)

triangle start shape (1, 2, 3)
(301, 2, 3)
[[[275.92 853.98   6.  ]
  [279.81 855.13   6.  ]]

 [[266.67 852.51   6.  ]
  [271.51 853.07   6.  ]]

 [[273.03 850.59   6.  ]
  [267.83 849.89   6.  ]]

 ...

 [[270.29 855.45   6.  ]
  [252.4  855.52   6.  ]]

 [[279.81 855.13   6.  ]
  [270.29 855.45   6.  ]]

 [[271.51 853.07   6.  ]
  [275.92 853.98   6.  ]]]


In [7]:
def shares_point(poly1,poly2):
  if(len(poly2.shape)>2):
    poly2 = np.squeeze(poly2)
  if(len(poly1.shape)>2):
    poly1 = np.squeeze(poly1)
  for i in range(poly1.shape[0]):
    for j in range(poly2.shape[0]):
      if(np.allclose(poly1[i], poly2[j],atol=0.01)):
          return True
  return False


def get_loops(lines):
	#used to track which lines are already "claimed"
	line_used = np.full(lines.shape[0], False)
	line_loops = []
	
	#until all lines have been used
	while not np.all(line_used == True):
		#initiates temporary loop for population
		temp_start_index =np.argmax(~line_used) #finds the first unclaimed line to seed next shape
		# print("new start index: " , temp_start_index)
		temp_start_line = lines[temp_start_index]
		line_used[temp_start_index] = True
		temp_loop = np.array([temp_start_line])
		temp_line = temp_start_line
		loop_closed = False
		#checking for next line in sequence
		while not loop_closed:
			for i in range(lines.shape[0]):
				#starts at last line start for speed
				wrapped_ind = (temp_start_index+i)%lines.shape[0]
				#checks if a point is shared with the last line
				if (not line_used[wrapped_ind]) and shares_point(lines[wrapped_ind], temp_line):
					# print("adding line ", wrapped_ind)
					line_used[wrapped_ind] = True
					temp_line = lines[wrapped_ind]
					temp_loop = np.vstack((temp_loop, [temp_line]))
					# print("loop dimension" , temp_loop.shape)

					#loop is closed
					#if not the starting line and shares a point with the starting line
					# if(not np.allclose(temp_line,  temp_start_line, atol=0.001 ) and shares_point(temp_line, temp_start_line)):
					if( temp_loop.shape[0]>2 and shares_point(temp_line, temp_start_line)):
							line_loops.append(temp_loop)
							# print("Loop closed! Now the numberof loops is ", len(line_loops))
							# print(line_loops[-1])
							loop_closed = True
							break
			# print("Failed to find adjacent line")
		
	return line_loops
					

loops = get_loops(surface_lines)

for i in range(len(loops)):
	print("loop ", i, ":")
	print("points: ",  loops[i].shape)
	for j in range(loops[i].shape[0]):
		print(loops[i][j])

	print("\n")
	


loop  0 :
points:  (13, 2, 3)
[[275.92 853.98   6.  ]
 [279.81 855.13   6.  ]]
[[279.81 855.13   6.  ]
 [270.29 855.45   6.  ]]
[[270.29 855.45   6.  ]
 [252.4  855.52   6.  ]]
[[252.4  855.52   6.  ]
 [242.78 855.89   6.  ]]
[[237.28 856.31   6.  ]
 [242.78 855.89   6.  ]]
[[237.28 851.82   6.  ]
 [237.28 856.31   6.  ]]
[[237.28 850.46   6.  ]
 [237.28 851.82   6.  ]]
[[237.28 850.46   6.  ]
 [239.43 850.48   6.  ]]
[[239.43 850.48   6.  ]
 [248.84 852.07   6.  ]]
[[248.84 852.07   6.  ]
 [252.86 852.38   6.  ]]
[[252.86 852.38   6.  ]
 [266.67 852.51   6.  ]]
[[266.67 852.51   6.  ]
 [271.51 853.07   6.  ]]
[[271.51 853.07   6.  ]
 [275.92 853.98   6.  ]]


loop  1 :
points:  (49, 2, 3)
[[273.03 850.59   6.  ]
 [267.83 849.89   6.  ]]
[[267.83 849.89   6.  ]
 [263.1  849.68   6.  ]]
[[263.1  849.68   6.  ]
 [253.97 849.71   6.  ]]
[[253.97 849.71   6.  ]
 [249.71 849.45   6.  ]]
[[249.71 849.45   6.  ]
 [239.89 847.8    6.  ]]
[[237.28 847.78   6.  ]
 [239.89 847.8    6.  ]]
[[237.2

In [8]:
#ensures that the points on each line is in the correct order

def fix_loop_order(loop):
	# checks if first point is incorrectly ordered.
	if np.allclose(loop[0,0,:], loop[1,0,:],atol=0.01) or np.allclose(loop[0,0,:], loop[1,1,:],atol=0.01):
		print("Switched first from ")
		print(loop[0,:,:])
		tempA = np.copy(loop[0,0,:])
		tempB = np.copy(loop[0,1,:])

		loop[0,0,:] = tempB
		loop[0,1,:] = tempA
		print("to:  ")
		print(loop[0,:,:])
		

	for line_ind in range(loop.shape[0]-1):
		#checks if the second point in the current segment is the same as the first point in the next segment. If not, switches it
		if not np.allclose(loop[line_ind,1,:], loop[line_ind+1,0,:],atol=0.01):
			tempA = np.copy(loop[line_ind+1,0,:])
			tempB = np.copy(loop[line_ind+1,1,:])
			
			loop[line_ind+1,0,:] = tempB
			loop[line_ind+1,1,:] = tempA
	return loop


for i in range(len(loops)):
	# print("loop ", i, ":")
	# print("points: ",  loops[i].shape[0])
	loops[i] = fix_loop_order(loops[i])

print("Updated loop orders: ")
for i in range(len(loops)):
	print("loop ", i, ":")
	print("points: ",  loops[i].shape[0])
	for j in range(loops[i].shape[0]):
		print(loops[i][j])

	print("\n")


	

Switched first from 
[[300.47 862.11   6.  ]
 [326.84 862.02   6.  ]]
to:  
[[326.84 862.02   6.  ]
 [300.47 862.11   6.  ]]
Switched first from 
[[270.02 932.35   6.  ]
 [262.59 933.41   6.  ]]
to:  
[[262.59 933.41   6.  ]
 [270.02 932.35   6.  ]]
Updated loop orders: 
loop  0 :
points:  13
[[275.92 853.98   6.  ]
 [279.81 855.13   6.  ]]
[[279.81 855.13   6.  ]
 [270.29 855.45   6.  ]]
[[270.29 855.45   6.  ]
 [252.4  855.52   6.  ]]
[[252.4  855.52   6.  ]
 [242.78 855.89   6.  ]]
[[242.78 855.89   6.  ]
 [237.28 856.31   6.  ]]
[[237.28 856.31   6.  ]
 [237.28 851.82   6.  ]]
[[237.28 851.82   6.  ]
 [237.28 850.46   6.  ]]
[[237.28 850.46   6.  ]
 [239.43 850.48   6.  ]]
[[239.43 850.48   6.  ]
 [248.84 852.07   6.  ]]
[[248.84 852.07   6.  ]
 [252.86 852.38   6.  ]]
[[252.86 852.38   6.  ]
 [266.67 852.51   6.  ]]
[[266.67 852.51   6.  ]
 [271.51 853.07   6.  ]]
[[271.51 853.07   6.  ]
 [275.92 853.98   6.  ]]


loop  1 :
points:  49
[[273.03 850.59   6.  ]
 [267.83 849.89   6. 

In [9]:
# conversion to polygons defined by points (not lines anymore)

def loop_to_poly(line_loop):
	poly = line_loop[0,0,:]
	for line_ind in range(1,line_loop.shape[0]):
		poly = np.vstack((poly, line_loop[line_ind,0,:]))
	return poly

# print(loops[0].shape)
# print(loops[0])
# print("------")
# print(loop_to_poly(loops[0]))

polys = []
for i in range(len(loops)):
	polys.append(loop_to_poly(loops[i]))

print(polys)

[array([[275.92, 853.98,   6.  ],
       [279.81, 855.13,   6.  ],
       [270.29, 855.45,   6.  ],
       [252.4 , 855.52,   6.  ],
       [242.78, 855.89,   6.  ],
       [237.28, 856.31,   6.  ],
       [237.28, 851.82,   6.  ],
       [237.28, 850.46,   6.  ],
       [239.43, 850.48,   6.  ],
       [248.84, 852.07,   6.  ],
       [252.86, 852.38,   6.  ],
       [266.67, 852.51,   6.  ],
       [271.51, 853.07,   6.  ]], dtype=float32), array([[273.03, 850.59,   6.  ],
       [267.83, 849.89,   6.  ],
       [263.1 , 849.68,   6.  ],
       [253.97, 849.71,   6.  ],
       [249.71, 849.45,   6.  ],
       [239.89, 847.8 ,   6.  ],
       [237.28, 847.78,   6.  ],
       [237.28, 832.43,   6.  ],
       [243.03, 832.2 ,   6.  ],
       [245.69, 831.85,   6.  ],
       [248.42, 831.  ,   6.  ],
       [251.07, 829.52,   6.  ],
       [253.51, 827.5 ,   6.  ],
       [255.27, 825.42,   6.  ],
       [256.47, 823.23,   6.  ],
       [258.1 , 822.83,   6.  ],
       [259.77, 822.03,  

In [10]:
#ensures that all output polygons are defined by points in clockwise order.
# based on  https://stackoverflow.com/questions/1165647/how-to-determine-if-a-list-of-polygon-points-are-in-clockwise-order#:~:text=Calculate%20the%20signed%20area%20of,it's%20positive%20they%20are%20counterclockwise.
def signedArea(x1,y1,x2,y2):
	return (x1*y2) - (x2*y1)

def is_clockwise(polygon):
	# print(polygon.shape)
	area = signedArea(polygon[-1,0], polygon[-1,1], polygon[0,0], polygon[0,1])
	for i in range(polygon.shape[0]-1):
		area = area+signedArea(polygon[i,0], polygon[i,1], polygon[i+1,0], polygon[i+1,1])
		#  If it's negative = clockwise order,  positive = counterclockwise.
	if area < 0:
		return True
	return False



	
for i in range(len(polys)):
	# ccw
	if not is_clockwise(polys[i]):
		print("flipping from : ")
		print(polys[i])
		polys[i] = np.flip(polys[i],0)
		print("to: ")
		print(polys[i])
		print("\n")
		
		

flipping from : 
[[275.92 853.98   6.  ]
 [279.81 855.13   6.  ]
 [270.29 855.45   6.  ]
 [252.4  855.52   6.  ]
 [242.78 855.89   6.  ]
 [237.28 856.31   6.  ]
 [237.28 851.82   6.  ]
 [237.28 850.46   6.  ]
 [239.43 850.48   6.  ]
 [248.84 852.07   6.  ]
 [252.86 852.38   6.  ]
 [266.67 852.51   6.  ]
 [271.51 853.07   6.  ]]
to: 
[[271.51 853.07   6.  ]
 [266.67 852.51   6.  ]
 [252.86 852.38   6.  ]
 [248.84 852.07   6.  ]
 [239.43 850.48   6.  ]
 [237.28 850.46   6.  ]
 [237.28 851.82   6.  ]
 [237.28 856.31   6.  ]
 [242.78 855.89   6.  ]
 [252.4  855.52   6.  ]
 [270.29 855.45   6.  ]
 [279.81 855.13   6.  ]
 [275.92 853.98   6.  ]]


flipping from : 
[[273.03 850.59   6.  ]
 [267.83 849.89   6.  ]
 [263.1  849.68   6.  ]
 [253.97 849.71   6.  ]
 [249.71 849.45   6.  ]
 [239.89 847.8    6.  ]
 [237.28 847.78   6.  ]
 [237.28 832.43   6.  ]
 [243.03 832.2    6.  ]
 [245.69 831.85   6.  ]
 [248.42 831.     6.  ]
 [251.07 829.52   6.  ]
 [253.51 827.5    6.  ]
 [255.27 825.42   6. 

In [11]:
# moving everything to the first quadrant.

def reset_origin_polys(polygons, x_offset=0.0, y_offset=0.0):
	min_x =  float("inf")
	min_y =  float("inf")
	# getting minimum indices
	for poly_ind in range(int(len(polygons))):
		# if(surface_tris[tri_ind].shape[0] != 3):
		# 	print("Shape is not a triangle! ", str(surface_tris[tri_ind].shape[0]), " Points detected!")
		# 	print(surface_tris[tri_ind])
		for point_ind in range(polygons[poly_ind].shape[0]):
			if polygons[poly_ind][point_ind,0] < min_x:
				min_x = polygons[poly_ind][point_ind,0] 
			if polygons[poly_ind][point_ind,1] < min_y:
				min_y = polygons[poly_ind][point_ind,1] 
	
	print()
	# updating all points
	for poly_ind in range(int(len(polygons))):

		
		for point_ind in range(polygons[poly_ind].shape[0]):
			if(min_x<0):
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]+abs(min_x)+x_offset
			else:
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]-abs(min_x)+x_offset

			if(min_y<0):
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]+abs(min_y)+y_offset
			else:
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]-abs(min_y)+y_offset


	return polygons

print(polys)
polygons = reset_origin_polys(polys,45,15)
print(polygons)


[array([[271.51, 853.07,   6.  ],
       [266.67, 852.51,   6.  ],
       [252.86, 852.38,   6.  ],
       [248.84, 852.07,   6.  ],
       [239.43, 850.48,   6.  ],
       [237.28, 850.46,   6.  ],
       [237.28, 851.82,   6.  ],
       [237.28, 856.31,   6.  ],
       [242.78, 855.89,   6.  ],
       [252.4 , 855.52,   6.  ],
       [270.29, 855.45,   6.  ],
       [279.81, 855.13,   6.  ],
       [275.92, 853.98,   6.  ]], dtype=float32), array([[277.71, 851.66,   6.  ],
       [282.01, 853.06,   6.  ],
       [285.45, 854.62,   6.  ],
       [293.54, 853.28,   6.  ],
       [297.41, 852.32,   6.  ],
       [301.08, 851.14,   6.  ],
       [306.17, 848.92,   6.  ],
       [308.39, 847.64,   6.  ],
       [310.4 , 846.26,   6.  ],
       [312.58, 844.43,   6.  ],
       [314.46, 842.49,   6.  ],
       [317.48, 838.37,   6.  ],
       [319.46, 834.76,   6.  ],
       [321.  , 831.1 ,   6.  ],
       [319.13, 827.66,   6.  ],
       [317.06, 824.47,   6.  ],
       [314.83, 821.52,  

In [12]:
# exporting to a file

num_polys = len(polygons)
# output_name = "neuro_model.txt"
# output_name = "squares.txt"
# output_name = "circle_split.txt"
# output_name = "simple_split.txt"
# output_name = "neuro_model_half.txt"
output_name = "neuro_model_half_rotated.txt"
# output_name = "vessel_quarter.txt"


with open(output_name, "wb") as f:
	f.write(str("num polygons\n").encode())
	f.write(str(str(num_polys) + "\n\n").encode())
	for poly_ind in range(int(num_polys)):
		f.write(str(polygons[poly_ind].shape[0]).encode())
		f.write(("\n").encode())
		for point_ind in range(polygons[poly_ind].shape[0]):
			np.savetxt(f,[polygons[poly_ind][point_ind,0:2]], fmt='%.2f',delimiter=',')
		f.write(b"\n")


-----------------------